In [1]:
import numpy as np
import pandas as pd
import networkx as nx
#import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go

In [2]:

data = pd.read_csv('/Users/andrei/PycharmProjects/TWITTER_F/data/SELECT_FINAL_F.csv',\
                   sep='\t', lineterminator ='\n')
data.head()

,id,conversation_id,user_id,username,tweet,photos,replies_count,retweets_count,likes_count,hashtags,video,thumbnail,parent_tweet_id,date_time,date_only,quote_url,link,reply_to_scr_name,reply_to_name,parentie
0,1349143722940919808,1169297966303260672,1169297966303260672,qod_gme,@MaryTravels78 If it’s any consolation ... it ...,0,0,0,0,[],0,0,1320196554821668864,2021-01-13 02:58:24+03:00,2021-01-13 00:00:00+03:00,NaN,https://twitter.com/QOD_GME/status/13491437229...,MaryTravels78,Mary Inge,1320196554821668865
1,1349143720847953923,376727981,376727981,trey50daniel,@jennkruza @RodAlzmann About to be the stimmy ...,0,0,0,8,[],0,0,26795056,2021-01-13 02:58:23+03:00,2021-01-13 00:00:00+03:00,NaN,https://twitter.com/Trey50Daniel/status/134914...,jennkruza,jenn kruza,26795056
2,1349143059120881664,1320196554821668864,1320196554821668864,marytravels78,"@QOD_GME Of not, there should be...",0,1,0,0,[],0,0,1169297966303260672,2021-01-13 02:55:45+03:00,2021-01-13 00:00:00+03:00,NaN,https://twitter.com/MaryTravels78/status/13491...,QOD_GME,QOD: Question of the Day,1169297966303260672
3,1349140350628220934,967494738369753216,967494738369753216,dantzfrye,I start my day off everyday with 0$ But I won’...,0,0,0,0,['gme'],0,0,0,2021-01-13 02:45:00+03:00,2021-01-13 00:00:00+03:00,NaN,https://twitter.com/DantzFrye/status/134914035...,NaN,NaN,0
4,1349138601058193410,83749263,83749263,tmyrbrgh,@investing_city About to to be $GME Gamestop,0,0,0,2,[],0,0,2961979644,2021-01-13 02:38:02+03:00,2021-01-13 00:00:00+03:00,NaN,https://twitter.com/tmyrbrgh/status/1349138601...,investing_city,Ryan Reeves,2961979644


In [3]:
data = data.drop(columns= ['tweet', 'photos',
       'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'video',
       'thumbnail'])

In [4]:
graph_base = data[data['reply_to_scr_name'].isnull() == False]

In [5]:
A = list(graph_base['username'].unique())

B = list(graph_base['reply_to_scr_name'].unique())

username_list = set(A+B)

len(username_list)

502010

In [6]:
l_id = [i for i in range(len(username_list))]

len(l_id)

502010

In [7]:
#node_list.remove(np.nan)

mapper = dict(zip(username_list, l_id))
r_mapper = dict(zip(l_id,username_list))

gr_data = pd.DataFrame()

gr_data['FromID'] = graph_base['username'].map(mapper)
gr_data['ToID'] = graph_base['reply_to_scr_name'].map(mapper)

In [8]:
gr_data.head()

,FromID,ToID
0,190486,234058
1,236606,107536
2,110667,432448
4,480733,51106
7,107536,312412


# Creating Graph

In [9]:
G = nx.Graph()

In [47]:
for i in l_id:
    G.add_node(i)

In [48]:
for i,j in gr_data.iterrows():
    G.add_edges_from([(j['FromID'],j["ToID"])])

In [ ]:
pos = nx.spring_layout(G, k=0.2, iterations=10)

In [ ]:
for n, p in pos.items():
    G.node[n]['pos'] = p

In [ ]:

edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

In [ ]:

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='RdBu',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0)))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])


In [ ]:

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>AT&T network connections',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="No. of connections",
                    showarrow=False,
                    xref="paper", yref="paper") ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(fig)
plotly.plot(fig)